In [1]:
### Basic module import
import os           # Working directory
import pandas as pd # Data processing
import numpy as np  # Scientific computing/matrix algebra
import matplotlib
import matplotlib.pyplot as plt # Common graphing interface (check also plotly and plotnine)
import re # Regular expressions

In [2]:
%cd ~/snap/snapd-desktop-integration/current/Documents/Thesis/data

/home/s212574/snap/snapd-desktop-integration/83/Documents/Thesis/data


In [3]:
### Reading the job data
job_df = pd.read_csv('job.csv', sep=',')

# Defining the 'categorical' and 'numerical' lists and changing the datatypes accordingly
categorical = ['Municipality', 'AgeGroup', 'Gender', 'Sector', 'Socio']
numerical   = ['Val', 'Year']

job_df[categorical] = job_df[categorical].astype('category')

# Saving this set for further investigation
job_mun = set(job_df['Municipality'])


In [4]:
%cd ~/snap/snapd-desktop-integration/current/Documents/Thesis/data

/home/s212574/snap/snapd-desktop-integration/83/Documents/Thesis/data


In [5]:
# Reading of the commuting matrix
#del job_mun, pop_mun
cm_df = pd.read_csv('commuter_codes.csv', sep=',')

# Defining the 'categorical' and 'numerical' lists and changing the datatypes accordingly
categorical = ['Gender', 'Residence', 'Work']
numerical = ['Year', 'value']

cm_df[categorical] = cm_df[categorical].astype('category')

# Commuter matrix with names instead of numbers
cm_df_val = pd.read_csv('commuter_values.csv', sep=',')

# Create a serial number to merge
cm_df['id'] = cm_df.index
cm_df_val['id'] = cm_df_val.index

# Merge the databases
cm_df_tot = cm_df.merge(cm_df_val, on='id', suffixes=('_c', '_v'))

# Convert the residence code for further work
cm_df_tot['Residence_c'] = cm_df_tot['Residence_c'].astype('int64')

names_codes = cm_df_tot[['Work_c', 'Work_v']].drop_duplicates()

In [6]:
cm_df_tot

,Unnamed: 0_c,Year_c,Gender_c,Work_c,Residence_c,value_c,id,Unnamed: 0_v,Year_v,Gender_v,Work_v,Residence_v,value_v
0,0,2018,M,270,2,242,0,0,2018,Men,Gribskov,Province Københavns omegn,242
1,1,2018,K,306,2,42,1,1,2018,Women,Odsherred,Province Københavns omegn,42
2,2,2018,M,306,2,93,2,2,2018,Men,Odsherred,Province Københavns omegn,93
3,3,2018,K,316,2,260,3,3,2018,Women,Holbæk,Province Københavns omegn,260
4,4,2018,M,316,2,390,4,4,2018,Men,Holbæk,Province Københavns omegn,390
...,...,...,...,...,...,...,...,...,...,...,...,...,...
296963,296963,2010,K,159,175,264,296963,296963,2010,Women,Gladsaxe,Rødovre,264
296964,296964,2010,M,159,175,333,296964,296964,2010,Men,Gladsaxe,Rødovre,333
296965,296965,2010,K,161,175,336,296965,296965,2010,Women,Glostrup,Rødovre,336
296966,296966,2010,M,161,175,294,296966,296966,2010,Men,Glostrup,Rødovre,294


In [7]:
# Read the population data
pop_df = pd.read_csv('pop.csv', sep=',')

# Defining the 'categorical' and 'numerical' lists and changing the datatypes accordingly
categorical = ['Municipality', 'edu', 'PopSocio', 'Sector', 'AgeGroup', 'Gender']

pop_df[categorical] = pop_df[categorical].astype('category')

pop_mun = set(pop_df['Municipality'])

pop_df['Gender'] = pop_df.Gender.cat.rename_categories({'Men': 'M', 'Women': 'K'})

pop_df = pd.merge(pop_df, names_codes, left_on='Municipality', right_on='Work_v')
pop_df[['edu_c', 'edu_v']] = pd.DataFrame(pop_df['edu'].str.split(" ", n=1, expand=True))
pop_df['PopSocio_c'] = pop_df.PopSocio.cat.rename_categories({'Enrolled in education': '0', 'Employed': '1', 'Unemployed':'2', 'Outside the labour force':'3'})


In [8]:
%cd ~/snap/snapd-desktop-integration/current/Documents/Thesis/data

/home/s212574/snap/snapd-desktop-integration/83/Documents/Thesis/data


In [9]:
# TU
ses_df = pd.read_csv('session.csv', sep=',') # 140672 obs

print(ses_df['RespPrimOcc'].unique().tolist())
# ses_df = pd.read_csv('session.txt', sep=';', encoding = "ISO-8859-1")

# Defining the 'categorical' and 'numerical' lists and changing the datatypes accordingly.

# PC
# categorical = ['RespSex', 'RespEdulevel', 'RespPrimOcc', 'HomeAdrMunCode', 'PrimOccMuncode', 'HousehAccomodation']

# Server
categorical = ['RespSex', 'RespEdulevel', 'RespPrimOcc', 'HomeAdrMunCode', 'PrimOccMuncode']
numerical = ['RespAgeCorrect', 'IncRespondent', 'SessionWeight', 'DiaryYear', 'TotalMin']

all_vars = categorical + numerical

#ses_df = ses_df[all_vars]
ses_df[categorical] = ses_df[categorical].astype('category')

# What are the values on home and occupation codes on the TU data
occ_cat = set(list(ses_df.PrimOccMuncode.cat.categories))
hom_cat = set(list(ses_df.HomeAdrMunCode.cat.categories))

# Which of these appear on the names_codes dataframe we created earlier?
diff = list(set(names_codes.Work_c) ^ occ_cat)
diff = [int(i) for i in diff]

names_codes.loc[names_codes['Work_c'].isin(diff)]

## At first, it seems that we dont need any of the pop and cm matrices we have. We could just aggregate the data on the TU
#   dataset and run the gibbs sampling on it. This needs further discussion.
print(ses_df['RespPrimOcc'].unique().tolist())
ses_df = ses_df.drop(ses_df[ses_df.PrimOccMuncode.isin(diff) | # Occupation municipality
                            ses_df.HomeAdrMunCode.isin(diff)].index) # Home municipality
print(ses_df['RespPrimOcc'].unique().tolist())

ses_df[categorical].describe()._append(ses_df[categorical].isnull().sum().rename('isnull'))


[107.0, 370.0, 210.0, 221.0, 120.0, 222.0, 320.0, 116.0, 310.0, 360.0, nan, 350.0, 130.0, 390.0, 211.0, 231.0, 232.0, 233.0, 10.0, 103.0]
[107.0, 370.0, 210.0, 221.0, 120.0, 222.0, 320.0, 116.0, 310.0, 360.0, nan, 350.0, 130.0, 390.0, 211.0, 231.0, 232.0, 233.0, 10.0, 103.0]
[107.0, 370.0, 210.0, 221.0, 120.0, 222.0, 320.0, 116.0, 310.0, 360.0, nan, 350.0, 130.0, 390.0, 211.0, 231.0, 232.0, 233.0, 10.0, 103.0]


,RespSex,RespEdulevel,RespPrimOcc,HomeAdrMunCode,PrimOccMuncode
count,212326,211912.0,210932.0,212326,150466.0
unique,2,11.0,19.0,99,99.0
top,2,11.0,210.0,101,101.0
freq,108590,46650.0,93804.0,20944,19062.0
isnull,0,414.0,1394.0,0,61860.0


In [10]:
# Na's from the session database
# Any variable
categorical = ['RespSex', 'RespEdulevel', 'RespPrimOcc', 'HomeAdrMunCode']
for var in categorical:
    print(var)
    ses_df.drop(ses_df[ses_df[var].isnull()].index, inplace=True)
    print(ses_df['RespPrimOcc'].unique().tolist())
# Uknown category from occupation
ses_df.drop(ses_df[ses_df['RespPrimOcc']==9.].index, inplace=True)

RespSex
[107.0, 370.0, 210.0, 221.0, 120.0, 222.0, 320.0, 116.0, 310.0, 360.0, nan, 350.0, 130.0, 390.0, 211.0, 231.0, 232.0, 233.0, 10.0, 103.0]
RespEdulevel
[107.0, 370.0, 210.0, 221.0, 120.0, 222.0, 320.0, 116.0, 310.0, 360.0, nan, 350.0, 130.0, 390.0, 211.0, 231.0, 232.0, 233.0, 10.0, 103.0]
RespPrimOcc
[107.0, 370.0, 210.0, 221.0, 120.0, 222.0, 320.0, 116.0, 310.0, 360.0, 350.0, 130.0, 390.0, 211.0, 231.0, 232.0, 233.0, 10.0, 103.0]
HomeAdrMunCode
[107.0, 370.0, 210.0, 221.0, 120.0, 222.0, 320.0, 116.0, 310.0, 360.0, 350.0, 130.0, 390.0, 211.0, 231.0, 232.0, 233.0, 10.0, 103.0]


In [11]:
  # Na's from the population database
pop_df.drop(pop_df[pop_df.edu=='H90'].index, inplace=True) # Non-stated education
pop_df.drop(pop_df[pop_df.Sector=='X'].index, inplace=True) # Non-stated activity
pop_df['Sector'] = pop_df.Sector.cat.remove_unused_categories() # Remove unused categories

# Recoding of categorical variables
#### Job
job_df['AgeGroup'] = np.where(job_df['AgeGroup']=='-15', '15-19', job_df['AgeGroup'])

# Turning it back again to category
job_df['AgeGroup'] = job_df['AgeGroup'].astype('category')

#### Pop
# Education
pop_df['edu'] = pop_df['edu'].astype(str)
pop_df['edu'][(pop_df['edu'] =='H70') | (pop_df['edu'] =='H80')] = 'H99'
pop_df['edu'] = pop_df['edu'].astype('category')

#### TU
# Age
age_bins = [-1, 19, 24, 29, 34, 39, 44, 49, 54, 59, 64, 200]
age_cats = list(pop_df.AgeGroup.cat.categories)

# Create the categorical variable
ses_df['AgeGroup'] = pd.cut(ses_df.RespAgeCorrect, age_bins, labels=age_cats)

# Education, still in process
ses_df['edu'] = ''
ses_df['edu'][(ses_df['RespEdulevel'] ==1.) | 
                    (ses_df['RespEdulevel']==2.) | 
                    (ses_df['RespEdulevel']==3.) | 
                    (ses_df['RespEdulevel']==4.)] = 'H10'
ses_df['edu'][ses_df['RespEdulevel'] ==5.] = 'H20'
ses_df['edu'][ses_df['RespEdulevel'] ==6.] = 'H50'
ses_df['edu'][ses_df['RespEdulevel'] ==9.] = 'H35'
ses_df['edu'][ses_df['RespEdulevel'] ==11.] = 'H30'
ses_df['edu'][ses_df['RespEdulevel'] ==12.] = 'H40'
ses_df['edu'][(ses_df['RespEdulevel'] ==13.) | (ses_df['RespEdulevel'] ==14.)] = 'H99'

ses_df['edu'].unique()

# Reset it to be a category
ses_df['edu'] = ses_df['edu'].astype('category')

# Main occupation of the respondent
#ses_df['PopSocio'] = ''
#ses_df['PopSocio'][(ses_df['RespPrimOcc'] ==1.) | 
#                    (ses_df['RespPrimOcc']==2.) | 
#                    (ses_df['RespPrimOcc']==3.)] = '0'
#ses_df['PopSocio'][(ses_df['RespPrimOcc'] ==22.) | 
#                    (ses_df['RespPrimOcc']==30.) | # ?
#                    (ses_df['RespPrimOcc']==50.) | # ?
#                    (ses_df['RespPrimOcc']==52.)] = '1' # ?
#ses_df['PopSocio'][(ses_df['RespPrimOcc'] ==11.)] = '2' # ?
#ses_df['PopSocio'][(ses_df['RespPrimOcc'] ==15.) | 
#                    (ses_df['RespPrimOcc']==20.) | # ?
#                    (ses_df['RespPrimOcc']==12.) | # ?
#                    (ses_df['RespPrimOcc']==10.)] = '3' # ?

# Main occupation of the respondent
ses_df['PopSocio'] = ''
ses_df['PopSocio'][(ses_df['RespPrimOcc'] ==103.) | 
                     (ses_df['RespPrimOcc']==107.) | 
                    (ses_df['RespPrimOcc']==116.) | 
                    (ses_df['RespPrimOcc']==120.) | 
                    (ses_df['RespPrimOcc']==130.)] = '0'
ses_df['PopSocio'][(ses_df['RespPrimOcc'] ==211.) | 
                    (ses_df['RespPrimOcc']==210.) | # ?
                    (ses_df['RespPrimOcc']==221.) |
                    (ses_df['RespPrimOcc']==231.) |
                    (ses_df['RespPrimOcc']==232.) |
                    (ses_df['RespPrimOcc']==233.) |# ?
                    (ses_df['RespPrimOcc']==222.)] = '1' # ?
ses_df['PopSocio'][(ses_df['RespPrimOcc'] ==310.)] = '2' # ?
ses_df['PopSocio'][(ses_df['RespPrimOcc'] ==320.) | 
                    (ses_df['RespPrimOcc']==390.) | # ?
                    (ses_df['RespPrimOcc']==360.) |
                    (ses_df['RespPrimOcc']==370.) |
                    (ses_df['RespPrimOcc']==350.)] = '3' # ?

ses_df['PopSocio'].unique()

# Reset it to be a category
ses_df['PopSocio'] = ses_df['PopSocio'].astype('category')

# RespSex, change the names of the categories
ses_df['RespSex'] = ses_df.RespSex.cat.rename_categories({1.: 'M', 2.: 'K'})


/tmp/ipykernel_712430/3512778768.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pop_df['edu'][(pop_df['edu'] =='H70') | (pop_df['edu'] =='H80')] = 'H99'
/tmp/ipykernel_712430/3512778768.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ses_df['edu'][(ses_df['RespEdulevel'] ==1.) |
/tmp/ipykernel_712430/3512778768.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ses_df['edu'][ses_df['RespEdulevel'] ==5.] = 'H20'
/tmp/ipyker

In [12]:
pop_df_all = pop_df
pop_df['PopSocio'] = pop_df['PopSocio_c']
pop_df['edu'] = pop_df['edu_c']
pop_df['Municipality'] = pop_df['Work_c']
pop_df = pop_df[['Municipality', 'edu', 'PopSocio', 'AgeGroup', 'Gender', 'Year', 'Sector','Val']]

In [13]:
# Data aggregation
# Pop database
pop_agg = ['MunicipalityOrigin', 'edu', 'PopSocio', 'AgeGroup', 'Gender', 'Year'] # Variables we group by

pop_df = pop_df.rename(index=str, columns={'Municipality':'MunicipalityOrigin'})
pop_df = pop_df.groupby(pop_agg + ['Sector'], as_index=False).sum()

# Change the names of the sesssion database in order to make an easier merge
ses_names = {
    'PrimOccMuncode':'MunicipalityDest',
    'HomeAdrMunCode':'MunicipalityOrigin',
    'RespSex':'Gender',
    'DiaryYear':'Year',
}
ses_df = ses_df.rename(index=str, columns=ses_names)

# Merge:
# We get the combinations totals
pop_df['sums'] = pop_df.groupby(pop_agg, as_index=False)['Val'].transform(sum)

# Replace the zero valued sums with 1s
pop_df['sums'][(pop_df['sums']==0.)] = 1

# We divide the values by the totals to get the percentages
pop_df['percent'] = pop_df.Val/pop_df.sums
pop_df = pop_df.drop(['Val', 'sums'], axis=1)

# Make copies of the databases and make the indices of the databases the common variables.
idx_list = ['MunicipalityOrigin', 'edu', 'PopSocio', 'AgeGroup', 'Gender', 'Year']

# We convert the data types of the variables that don't have the same type, otherwise, merging won't be possible.
for var in ['MunicipalityOrigin', 'edu', 'PopSocio', 'AgeGroup', 'Gender']: 
    pop_df[var] = pop_df[var].astype(str)
    pop_df[var] = pop_df[var].astype('category')
    ses_df[var] = ses_df[var].astype(str)
    ses_df[var] = ses_df[var].astype('category')
    
ses_df_c = ses_df[(ses_df.Year>2008) & (ses_df.Year<2022)].set_index(idx_list)
pop_df_c = pop_df.copy().set_index(idx_list)

# Merge the databases so we have a distribution of industries for each person on TU
merged = ses_df_c.join(pop_df_c, how='inner')


# Drop unnecesary variables
merged = merged[['SessionId', 'Sector', 'percent']].reset_index(drop=True)

# We define the sampling function. This function is necessary when we want to aggregate the data by index
def groupby_sample(x):
    if all(x.percent==0.): # Just in case all of them are 0's
        return np.random.choice(x.Sector)
    else: 
        return np.random.choice(x.Sector, p=x.percent)

# This is the final dataframe. There are as many observations as the original TU data, but in this particular database, we have industries for each person too.
#samp_df = merged.groupby('SessionId', as_index=False).aggregate(groupby_sample)
samp_df = merged.groupby('SessionId', as_index=False).apply(groupby_sample).reset_index()
samp_df = samp_df.drop(['index'], axis=1)
samp_df.columns = ['SessionId', 'Sector']


samp_df = samp_df.merge(ses_df_c.reset_index(), on='SessionId', how='inner')

/tmp/ipykernel_712430/2504618920.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pop_df = pop_df.groupby(pop_agg + ['Sector'], as_index=False).sum()
/tmp/ipykernel_712430/2504618920.py:19: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pop_df['sums'] = pop_df.groupby(pop_agg, as_index=False)['Val'].transform(sum)
/tmp/ipykernel_712430/2504618920.py:19: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pop_df['sums'] = pop_df.g

In [14]:
samp_df

,SessionId,Sector,MunicipalityOrigin,edu,PopSocio,AgeGroup,Gender,Year,InterviewType,DiaryDate,...,JstartMuncode,JstartGMMzone,JstartFareZone,JstartNearestStation,JstartDistNearestStation,DayJourneyType,DayPrimTargetMuncode,DayPrimTargetPurp,SessionWeight,WeightOver6
0,141381,Activity not stated,540,H30,3,65-69 years,M,2009,0,14245,...,NaN,NaN,NaN,NaN,NaN,2,540.0,41.0,345.950094,NaN
1,141385,Wholesale and retail trade,219,H30,1,45-49 years,K,2009,2,14245,...,219.0,219066.0,NaN,Skævinge,1.0,12,270.0,11.0,225.445616,NaN
2,141387,Construction,259,H50,1,40-44 years,K,2009,2,14245,...,259.0,259037.0,NaN,Ølby,4.3,21,259.0,41.0,268.293655,NaN
3,141388,Human health activities,849,H50,1,30-34 years,M,2009,2,14245,...,849.0,849214.0,NaN,Lindholm,23.9,21,849.0,41.0,169.162727,NaN
4,141392,Activity not stated,657,H10,3,65-69 years,K,2009,2,14245,...,NaN,NaN,NaN,NaN,NaN,1,657.0,1.0,359.475638,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107213,491403,Activity not stated,851,H20,0,20-24 years,K,2021,1,18845,...,851.0,851711.0,NaN,Aalborg,1.1,12,751.0,41.0,218.593107,213.857017
107214,492046,Wholesale and retail trade,773,H10,0,15-19 years,K,2018,1,17861,...,773.0,773021.0,NaN,Thisted,20.4,11,773.0,12.0,67.251837,67.279810
107215,492153,Activity not stated,210,H10,3,65-69 years,K,2017,1,17513,...,210.0,210014.0,NaN,Kokkedal,1.6,11,223.0,33.0,691.723248,690.727223
107216,492359,"Agriculture, forestry and fishing",787,H30,1,55-59 years,M,2012,1,15372,...,787.0,787215.0,NaN,Hørdum,0.4,11,787.0,61.0,697.438411,NaN


In [15]:
# First dataset
samp_df.to_csv('sampling_df.txt', sep=',', index=False)
samp_df = pd.read_csv('sampling_df.txt', sep=',')


# After analayzing the correlation between variables, variables with more than 95% of correlation were dropped from the database. This, hopefully will reduce the dimensionality of the input

#drop = ['JstartMuncode', 'JstartNTMzone', 'HomeAdrNTMzone', 'SduMuncode', 'SduOldMuncode', 'HomeAdrOldMuncode', 'DayStartOldMuncode', 'SduNTMzone', 'PrimOccNTMzone', 
#'PrimOccOldMuncode', 'IncNuclFamily2000', 'IncFamily2000', 'IncFamily', 'DayStartMuncode', 'DayStartNTMzone', 'IncHouseh2000', 'IncNuclFamily', 'IncHouseh', 
#'DayPrimTargetMuncode', 'DiaryDate', 'FamNumPers', 'HousehNumDrivLic', 'HousehNumAdults', 'FamNumPers1084', 'RespAgeSimple', 'HousehNumPers', 'IncRespondent', 
#'IncSpouse2000', 'RespYearBorn', 'NuclFamNumPers', 'IncSpouse', 'TotalLen', 'NuclFamNumPers1084', 'RespPrimOcc', 'RespEdulevel'] 

drop = ['JstartMuncode', 'SduMuncode', 'IncNuclFamily2000', 'IncFamily2000', 'IncFamily', 'DayStartMuncode', 'IncHouseh2000', 'IncNuclFamily', 'IncHouseh', 
'DayPrimTargetMuncode', 'DiaryDate', 'FamNumPers', 'HousehNumDrivLic', 'HousehNumAdults', 'FamNumPers1084', 'RespAgeSimple', 'HousehNumPers', 'IncRespondent', 
'IncSpouse2000', 'RespYearBorn', 'NuclFamNumPers', 'IncSpouse', 'TotalLen', 'NuclFamNumPers1084', 'RespPrimOcc', 'RespEdulevel'] 

samp_df.drop(drop, inplace=True, axis=1)

### Numerical variables define
numerical = ['IncRespondent2000', 'TotalLenExclComTrans', 'TotalMotorLen', 'TotalBicLen', 'TotalMin', 'TotalMotorMin', 'SessionId', 'WorkHoursPw', 'GISdistHW', 'HousehNumPers1084',
            'TotalNumTrips', 'NumTripsCorr', 'NumTripsExclComTrans', 'SessionWeight', 'HomeAdrCitySize', 'HomeAdrDistNearestStation', 'HwDayspW', 'WorkatHomeDayspM', 
             'JstartDistNearestStation', 'NightsAway'] # Numerical variables with missing values 
samp_df[numerical] = samp_df[numerical].astype('float64')

def replace_missing(df, grouping_vars = ['MunicipalityOrigin', 'AgeGroup', 'edu', 'PopSocio']):
    """
    Replaces the missing values of numerical variables on the dataframe by taking the mean (or could be median) of the variables grouped by certain variables.
    It makes a two dataframes: one with the nans which we use to replace the missing values and one with the rest of the variables. 
    """
    # Variables we groupby 
    numerical = ['IncRespondent2000', 'TotalLenExclComTrans', 'TotalMotorLen', 'TotalBicLen', 'TotalMin', 'TotalMotorMin', 'WorkHoursPw', 'GISdistHW', 'HousehNumPers1084',
            'TotalNumTrips', 'NumTripsCorr', 'NumTripsExclComTrans', 'SessionWeight', 'HomeAdrCitySize', 'HomeAdrDistNearestStation', 'HwDayspW', 'WorkatHomeDayspM', 
             'JstartDistNearestStation', 'NightsAway'] # Numerical variables with missing values 
    
    # Divide the dataset
    df_auxiliar = df.drop(numerical, axis=1) # Dataframe which we don't modify
    df_missing  = df[grouping_vars+numerical+['SessionId']] # Dataframe with missings

    # NaN replacement
    f = lambda x: x.fillna(np.random.choice(x)) # Function to fill the na values using a random element 
    df_missing = df_missing.groupby(grouping_vars).transform(f) # Applying the function to the grouping
    
    return df_missing.merge(df_auxiliar, on='SessionId')


print(len(samp_df))

samp_df = replace_missing(samp_df)
samp_df = replace_missing(samp_df, grouping_vars = ['MunicipalityOrigin', 'edu', 'PopSocio'])
samp_df = replace_missing(samp_df, grouping_vars = ['MunicipalityOrigin', 'PopSocio'])
samp_df = replace_missing(samp_df, grouping_vars = ['PopSocio'])
samp_df[numerical] = samp_df[numerical].fillna(samp_df[numerical].mean())
samp_df[numerical] = samp_df[numerical].astype('int32')

# Drop AgeGroup since we don't need it anymore
samp_df.drop(['AgeGroup'], inplace=True, axis=1)

samp_df = samp_df.dropna(thresh=0.2*len(samp_df), axis=1) # Drop columns with more than 20% of missing values

### Categorical variables define 
categorical = [col for col in list(samp_df) if col not in numerical]

samp_df[categorical] = samp_df[categorical].astype('category')
# Convert categorical to categorical data type
for cat in categorical:
    samp_df[cat] = samp_df[cat].cat.rename_categories({'nan': 'MISS'})
samp_df[categorical] = samp_df[categorical].astype('str')
samp_df[categorical].replace(to_replace=['NaN', 'nan'], value='MISS') # Replace NaN with MISS as a new category on categorical data so we don't have a problem in keras.
samp_df[categorical] = samp_df[categorical].astype('category')

# Second dataset
samp_df.to_csv('sampling_df_no_nan.txt', sep=',', index=False)

107218


In [17]:
samp_df.columns

IncRespondent2000
TotalLenExclComTrans
TotalMotorLen
TotalBicLen
TotalMin
TotalMotorMin
WorkHoursPw
GISdistHW
HousehNumPers1084
TotalNumTrips
NumTripsCorr
NumTripsExclComTrans
SessionWeight
HomeAdrCitySize
HomeAdrDistNearestStation
HwDayspW
WorkatHomeDayspM
JstartDistNearestStation
NightsAway
SessionId
Sector
MunicipalityOrigin
edu
PopSocio
Gender
Year
InterviewType
PseudoYear
DiaryMonth
DiaryWeekday
DiaryDaytype
HomeAdrNUTS
HomeAdrCityCode
HomeAdrGMMzone
HomeAdrFareZone
HomeAdrNearestStation
HomeParkPoss
RespAgeCorrect
PrimOccNUTS
MunicipalityDest
PrimOccGMMzone
PrimOccFareZone
WorkHourType
WorkPubPriv
SduNUTS
SduGMMzone
WorkParkPoss
RespHasBicycle
RespHasSeasonticket
RespHasRejsekort
ResphasDrivlic
RespDrivlicYear
RespIsmemCarshare
HousehNumcars
HousehCarOwnership
Handicap
HousehAccomodation
HousehAccOwnorRent
NuclFamType
PosInFamily
NuclFamNumAdults
NuclFamNumPersO6
NuclFamNumDrivLic
FamNumAdults
FamNumPersO6
FamNumDrivLic
HousehNumPersO6
DayStartNUTS
DayStartCityCode
DayStartGMMzon